In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# For progress bar
!pip install tqdm


In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm


In [ ]:
class SRDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None):
        self.lr_paths = sorted([os.path.join(lr_dir, f) for f in os.listdir(lr_dir)])
        self.hr_paths = sorted([os.path.join(hr_dir, f) for f in os.listdir(hr_dir)])
        self.transform = transform or transforms.ToTensor()

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr = Image.open(self.lr_paths[idx]).convert("RGB")
        hr = Image.open(self.hr_paths[idx]).convert("RGB")
        return self.transform(lr), self.transform(hr)


In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        def CBR(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
            )

        self.enc1 = CBR(3, 64)
        self.enc2 = CBR(64, 128)
        self.enc3 = CBR(128, 256)
        self.pool = nn.MaxPool2d(2)

        self.dec3 = CBR(256, 128)
        self.dec2 = CBR(128, 64)
        self.final = nn.Conv2d(64, 3, 1)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

    def forward(self, x):
        x1 = self.enc1(x)                      # 256x256 → 256x256
        x2 = self.enc2(self.pool(x1))          # 256x256 → 128x128
        x3 = self.enc3(self.pool(x2))          # 128x128 → 64x64

        x = self.upsample(x3)                  # 64x64 → 128x128
        x = self.dec3(x)                       # → 128x128

        x = self.upsample(x)                   # 128x128 → 256x256
        x = self.dec2(x)                       # → 256x256

        x = self.final(x)                      # final output: 256x256
        return x


In [ ]:
BATCH_SIZE = 16
PATCH_SIZE = 256

train_dataset = SRDataset(
    lr_dir="/kaggle/input/intel-dataset/dataset/train/LR",
    hr_dir="/kaggle/input/intel-dataset/dataset/train/HR"
)



train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet().to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
EPOCHS = 10  # Change as needed

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for lr, hr in loop:
        lr, hr = lr.to(device), hr.to(device)

        output = model(lr)
        loss = criterion(output, hr)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_loader):.4f}")


In [ ]:
torch.save(model.state_dict(), "unet_sr.pth")
print("✅ Model saved!")


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# --- UNet Model ---
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True)
        )
        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 3, 3, padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

# --- Dataset ---
class DistillationDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, teacher_dir):
        self.lr_paths = sorted([os.path.join(lr_dir, f) for f in os.listdir(lr_dir) if f.endswith('.png')])
        self.hr_paths = sorted([os.path.join(hr_dir, f) for f in os.listdir(hr_dir) if f.endswith('.png')])
        self.teacher_paths = sorted([os.path.join(teacher_dir, f) for f in os.listdir(teacher_dir) if f.endswith('.png')])
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr = self.transform(Image.open(self.lr_paths[idx]).convert("RGB"))
        hr = self.transform(Image.open(self.hr_paths[idx]).convert("RGB"))
        teacher = self.transform(Image.open(self.teacher_paths[idx]).convert("RGB"))
        return lr, hr, teacher

# --- Training Function ---
def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # ✅ CHANGE THESE PATHS TO YOUR DATASET NAMES
    lr_dir="/kaggle/input/intel-dataset/dataset/train/LR"
    hr_dir="/kaggle/input/intel-dataset/dataset/train/HR"
    teacher_dir = "/kaggle/input/d/lohithakakumani/teacher-output/teacher_resized"

    dataset = DistillationDataset(lr_dir, hr_dir, teacher_dir)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = UNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    l1_loss = nn.L1Loss()

    # You can adjust weights for pixel and distillation losses
    alpha = 1.0   # weight for pixel loss
    beta = 0.5    # weight for distillation loss

    num_epochs = 15

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for lr, hr, teacher in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            lr, hr, teacher = lr.to(device), hr.to(device), teacher.to(device)

            output = model(lr)
            pixel_loss = l1_loss(output, hr)
            distill_loss = l1_loss(output, teacher)
            loss = alpha * pixel_loss + beta * distill_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

    torch.save(model.state_dict(), "distilled_unet.pth")
    print("✅ Student model with distillation saved as 'distilled_unet.pth'")

# --- Run the training ---
if __name__ == "__main__":
    train_model()


In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# --- UNet Model ---
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True)
        )
        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 3, 3, padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x

# --- Dataset ---
class DistillationDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, teacher_dir):
        self.lr_paths = sorted([os.path.join(lr_dir, f) for f in os.listdir(lr_dir) if f.endswith('.png')])
        self.hr_paths = sorted([os.path.join(hr_dir, f) for f in os.listdir(hr_dir) if f.endswith('.png')])
        self.teacher_paths = sorted([os.path.join(teacher_dir, f) for f in os.listdir(teacher_dir) if f.endswith('.png')])
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr = self.transform(Image.open(self.lr_paths[idx]).convert("RGB"))
        hr = self.transform(Image.open(self.hr_paths[idx]).convert("RGB"))
        teacher = self.transform(Image.open(self.teacher_paths[idx]).convert("RGB"))
        return lr, hr, teacher

# --- Training Function ---
def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # ✅ CHANGE THESE PATHS TO YOUR DATASET NAMES
    lr_dir="/kaggle/input/intel-dataset/dataset/train/LR"
    hr_dir="/kaggle/input/intel-dataset/dataset/train/HR"
    teacher_dir = "/kaggle/input/d/lohithakakumani/teacher-output/teacher_resized"

    dataset = DistillationDataset(lr_dir, hr_dir, teacher_dir)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = UNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    l1_loss = nn.L1Loss()

    # You can adjust weights for pixel and distillation losses
    alpha = 1.0   # weight for pixel loss
    beta = 0.5    # weight for distillation loss

    num_epochs = 15

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for lr, hr, teacher in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            lr, hr, teacher = lr.to(device), hr.to(device), teacher.to(device)

            output = model(lr)
            pixel_loss = l1_loss(output, hr)
            distill_loss = l1_loss(output, teacher)
            loss = alpha * pixel_loss + beta * distill_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

    torch.save(model.state_dict(), "distilled_unet.pth")
    print("✅ Student model with distillation saved as 'distilled_unet.pth'")

# --- Run the training ---
if __name__ == "__main__":
    train_model()


Epoch 1/15: 100%|██████████| 100/100 [01:55<00:00,  1.15s/it]


Epoch 1, Loss: 0.2292


Epoch 2/15: 100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


Epoch 2, Loss: 0.0732


Epoch 3/15: 100%|██████████| 100/100 [01:46<00:00,  1.06s/it]


...

Epoch 13, Loss: 0.0417


Epoch 14/15: 100%|██████████| 100/100 [01:46<00:00,  1.06s/it]


Epoch 14, Loss: 0.0412


Epoch 15/15: 100%|██████████| 100/100 [01:47<00:00,  1.07s/it]

Epoch 15, Loss: 0.0406
✅ Student model with distillation saved as 'distilled_unet.pth'
